# Proyecto TELECOM Interconnect

##  Objetivo
La compañía de telecomunicaciones ‘ TELECOM Interconnect ’ busca poder pronosticar su tasa de cancelación de clientes, con la finalidad de ofrecer códigos promocionales y opciones de planes especiales en caso de conocer que un cliente pudiera dejar la compañía.

##  Recursos
El equipo de marketing de TELECOM proporciona la información que ha recopilado de algunos de sus clientes, y nos proporciona dicha información en 4 archivos.

##  Descripción de los datos
Los datos consisten en archivos obtenidos de diferentes fuentes y se describe a continuación el contenido generico de cada archivo:

    - contract.csv — información del contrato
    - personal.csv — datos personales del cliente
    - internet.csv — información sobre los servicios de Internet
    - phone.csv — información sobre los servicios telefónicos
    
Nota: En cada archivo, la columna customerID (ID de cliente) contiene un código único asignado a cada cliente. La información del contrato es válida a partir del 1 de febrero de 2020

###  EDA

In [1]:
# importación de librerias para EDA

import pandas as pd
import re

In [2]:
# carga de dataframes

contract = pd.read_csv('/datasets/final_provider/contract.csv', parse_dates=['EndDate'])
personal = pd.read_csv('/datasets/final_provider/personal.csv')
internet = pd.read_csv('/datasets/final_provider/internet.csv')
phone = pd.read_csv('/datasets/final_provider/phone.csv')

#### contract

In [3]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


In [4]:
# defino función para valores únicos por columna

def unique_values(dataframe):
    for column in dataframe.columns:
        unique_values = dataframe[column].unique()
        print(f"Valores únicos en la columna '{column}':")
        print(unique_values)
        print()

In [5]:
unique_values(contract)

Valores únicos en la columna 'customerID':
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']

Valores únicos en la columna 'BeginDate':
['2020-01-01' '2017-04-01' '2019-10-01' '2016-05-01' '2019-09-01'
 '2019-03-01' '2018-04-01' '2019-04-01' '2017-07-01' '2014-12-01'
 '2019-01-01' '2018-10-01' '2015-04-01' '2015-09-01' '2018-01-01'
 '2014-05-01' '2015-10-01' '2014-03-01' '2018-05-01' '2019-11-01'
 '2019-02-01' '2016-01-01' '2017-08-01' '2015-12-01' '2014-02-01'
 '2018-06-01' '2019-12-01' '2017-11-01' '2019-06-01' '2016-04-01'
 '2017-02-01' '2018-12-01' '2014-04-01' '2018-09-01' '2014-11-01'
 '2016-07-01' '2015-02-01' '2018-07-01' '2014-08-01' '2016-03-01'
 '2018-08-01' '2014-10-01' '2015-06-01' '2016-08-01' '2019-05-01'
 '2017-03-01' '2016-02-01' '2017-09-01' '2014-09-01' '2017-12-01'
 '2016-12-01' '2017-06-01' '2015-05-01' '2016-10-01' '2016-09-01'
 '2019-08-01' '2019-07-01' '2017-05-01' '2017-10-01' '2014-07-01'
 '2018-03-01' '2015-01-01' '2018-11-0

In [6]:
contract.describe()

,MonthlyCharges
count,7043.000000
mean,64.761692
std,30.090047
min,18.250000
25%,35.500000
50%,70.350000
75%,89.850000
max,118.750000


In [7]:
contract.isnull().sum()

customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64

In [8]:
print(f'El número total de filas duplicadas en este archivo es de {contract.duplicated().sum()} filas.')

El número total de filas duplicadas en este archivo es de 0 filas.


In [9]:
contract.head(10)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65
5,9305-CDSKC,2019-03-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,99.65,820.5
6,1452-KIOVK,2018-04-01,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4
7,6713-OKOMC,2019-04-01,No,Month-to-month,No,Mailed check,29.75,301.9
8,7892-POOKP,2017-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,104.80,3046.05
9,6388-TABGU,2014-12-01,No,One year,No,Bank transfer (automatic),56.15,3487.95


#### personal

In [10]:
personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [11]:
unique_values(personal)

Valores únicos en la columna 'customerID':
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']

Valores únicos en la columna 'gender':
['Female' 'Male']

Valores únicos en la columna 'SeniorCitizen':
[0 1]

Valores únicos en la columna 'Partner':
['Yes' 'No']

Valores únicos en la columna 'Dependents':
['No' 'Yes']



In [12]:
personal.isnull().sum()

customerID       0
gender           0
SeniorCitizen    0
Partner          0
Dependents       0
dtype: int64

In [13]:
print(f'El número total de filas duplicadas en este archivo es de {personal.duplicated().sum()} filas.')

El número total de filas duplicadas en este archivo es de 0 filas.


In [14]:
personal.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No
5,9305-CDSKC,Female,0,No,No
6,1452-KIOVK,Male,0,No,Yes
7,6713-OKOMC,Female,0,No,No
8,7892-POOKP,Female,0,Yes,No
9,6388-TABGU,Male,0,No,Yes


#### internet

In [15]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [16]:
unique_values(internet)

Valores únicos en la columna 'customerID':
['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']

Valores únicos en la columna 'InternetService':
['DSL' 'Fiber optic']

Valores únicos en la columna 'OnlineSecurity':
['No' 'Yes']

Valores únicos en la columna 'OnlineBackup':
['Yes' 'No']

Valores únicos en la columna 'DeviceProtection':
['No' 'Yes']

Valores únicos en la columna 'TechSupport':
['No' 'Yes']

Valores únicos en la columna 'StreamingTV':
['No' 'Yes']

Valores únicos en la columna 'StreamingMovies':
['No' 'Yes']



In [17]:
internet.isnull().sum()

customerID          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64

In [18]:
print(f'El número total de filas duplicadas en este archivo es de {internet.duplicated().sum()} filas.')

El número total de filas duplicadas en este archivo es de 0 filas.


In [19]:
internet.head(10)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No
5,9305-CDSKC,Fiber optic,No,No,Yes,No,Yes,Yes
6,1452-KIOVK,Fiber optic,No,Yes,No,No,Yes,No
7,6713-OKOMC,DSL,Yes,No,No,No,No,No
8,7892-POOKP,Fiber optic,No,No,Yes,Yes,Yes,Yes
9,6388-TABGU,DSL,Yes,Yes,No,No,No,No


#### phone

In [20]:
phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [21]:
unique_values(phone)

Valores únicos en la columna 'customerID':
['5575-GNVDE' '3668-QPYBK' '9237-HQITU' ... '2234-XADUH' '8361-LTMKD'
 '3186-AJIEK']

Valores únicos en la columna 'MultipleLines':
['No' 'Yes']



In [22]:
phone.isnull().sum()

customerID       0
MultipleLines    0
dtype: int64

In [23]:
print(f'El número total de filas duplicadas en este archivo es de {phone.duplicated().sum()} filas.')

El número total de filas duplicadas en este archivo es de 0 filas.


In [24]:
phone.head(10)

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes
5,7892-POOKP,Yes
6,6388-TABGU,No
7,9763-GRSKD,No
8,7469-LKBCI,No
9,8091-TTVAX,Yes


###  Observaciones
Descripción de los datos presentes en los archivos proporcionados por la compañia:

contract — información del contrato (7043 observaciones):

    - customerID
    - BeginDate (2013…2020)
    - EndDate (no | (fecha YYYY/MM/DD))
    - Type (?? Month-to-month | One Year | Two Year)
    - PaperlessBilling (No | Yes)
    - PaymentMethod (Bank transfer (automatic) | Credit card (automatic) | Electronic check | Mailed check)
    - MonthlyCharges ( .)
    - 𝑇𝑜𝑡𝑎𝑙𝐶ℎ𝑎𝑟𝑔𝑒𝑠( .$$)

internet — información sobre los servicios de Internet (5517 observaciones):

    - customerID
    - InternetServices (DLS | Fiber optic)
    - OnlineSecurity (Yes | no)
    - OnlineBackup (Yes | no)
    - DeviceProtect (Yes | no)
    - TechSupport (Yes | no)
    - StreamingTV (Yes | no)
    - StreamingMovies (Yes | no)

personal — datos personales del cliente (7043 observaciones):

    - gender (female | male)
    - SeniorCitizen (1 | 0)
    - Partner (Yes | no)
    - Dependents (Yes | no)

phone — información sobre los servicios telefónicos (6361 observaciones)

    - customerID
    - MultipleLines (Yes/No)
Se sabe que el tipo de servicios proporcionados por la compañía son:

Comunicación por teléfono fijo. El teléfono se puede conectar a varias líneas de manera simultánea.
Internet. La red se puede configurar a través de una línea telefónica (DSL, línea de abonado digital) o a través de un cable de fibra óptica.

Algunos otros servicios que ofrece la empresa incluyen:

Seguridad en Internet: software antivirus (ProtecciónDeDispositivo) y un bloqueador de sitios web maliciosos (SeguridadEnLínea).
Una línea de soporte técnico (SoporteTécnico).
Almacenamiento de archivos en la nube y backup de datos (BackupOnline).
Streaming de TV (StreamingTV) y directorio de películas (StreamingPelículas)

Característica objetivo: la columna 'EndDate' es igual a 'No'.

##  Plan de trabajo
Recordando que el objetivo es crear un modelo capaz de pronosticar la posible salida de un cliente, y con base en los documentos proporcionados por la compañía, se plantea el siguiente plan de trabajo:

###  EDA
Exploración de datos para comprender la naturaleza y el alcance de la información disponible. En esta sección se podrá considerar:

- Análisis de correlación de datos
- Identificación de datos atípicos
- Distribución de variables
- Entre otros que se consideren necesarios durante la exploración

###  Selección y adaptación de información
Se seleccionarán las características más relevantes de los datasets, y se adaptarán para su uso en un modelo de regresión logística. Este proceso incluirá:

- Selección de características
En caso de necesitarlo, se aplicarán técnicas de preparación de datos como:

- Equilibrio de clases
- Escalar características numéricas
- Codificación de variables categóricas
- Entre otros
Al finalizar esta sección, se reunirán todas las características seleccionadas y adaptadas, a un solo dataset, con el cual se pasará a la siguiente fase.

###  Elaboración y selección de modelo
Con el dataset previamente preparado, se desarrollarán y evaluarán varios modelos de Machine Learning para determinar cuál es el más adecuado para el objetivo. Este proceso incluirá:

- Entrenamiento de diferentes modelos
- Optimización de hiperparámetros
- Medición de resultados a través de la métrica de error cuadrático medio (MSE)
- Selección de modelo final

###  Ejecución del modelo seleccionado
Ya con el modelo elegido, se pondrá a prueba con el conjunto de validación, y se manejará como métrica de evaluación principal el AUC-ROC y como métrica adicional la exactitud.

###  Preparación del informe
Con el modelo y los resultados ya obtenidos, se realizará un informe detallado para la compañía, donde se proporcionará un resumen de nuestro trabajo, la razón de nuestras decisiones para crear el cuerpo del proyecto y cualquier recomendación que se pueda realizar para mejorar la retención de clientes.

Como nota adicional, se comenta que se tiene pensado trabajar el dataset final de la siguiente manera:

- 60% entrenamiento
- 20% prueba
- 20% validación

##  Preguntas
Revisando los datasets, existe algo que no queda claro:

- Dataset contract
    - MonthlyCharges ($.$$)
        - TotalCharges ($.$$)
En esta parte no me queda claro la razón para cada columna. Me doy la idea de que está relacionado con la columna 'type', pero de igual manera ¿Podrías proporcionar más detalles sobre el significado y la relación entre las columnas MonthlyCharges y TotalCharges en el dataset de contratos? Por citar un ejemplo para ser más especificos:

'|One year | No | Mailed check | 56.95 | 1889.5|'

el contrato es por 1 año, el cargo mensual es '56.95' y el total de cargos es '1889.5', no comprendo la relación, si el cargo mensual fuera 56.95 y si se multiplicara por 12, no daría el monto que se ve en 'TotalCharges' de 1889.5.

Por otra parte, no sé si esto se pueda preguntar en esta sección, pero me viene una pregunta constante y es ¿qué tanto peso podrá tener el trabajar con algunas de las características seleccionadas? ¿O deberé trabajar con todas?¿Alguna sugerencia?

Si existe alguna información adicional, me encantaría conocerla.

Gracias

Salta a la vista que la columna 'TotalCharges' de 'contract', se muestra como un dato tipo "objetct" cuando deberían ser identificado como una columnas con cantidades númericas.

In [25]:
non_numeric_values = contract['TotalCharges'].str.contains(r'\D', regex=True)
print(non_numeric_values.unique())

[ True False]


In [26]:
contract['TotalCharges'] = pd.to_numeric(contract['TotalCharges'], errors='coerce')

In [27]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7032 non-null   float64
dtypes: float64(2), object(6)
memory usage: 440.3+ KB


In [28]:
contract.isnull().sum()

customerID           0
BeginDate            0
EndDate              0
Type                 0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
dtype: int64

In [29]:
null_data_nan = contract[contract['TotalCharges'].isna()]

display(null_data_nan)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
488,4472-LVYGI,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
753,3115-CZMZD,2020-02-01,No,Two year,No,Mailed check,20.25,NaN
936,5709-LVOEQ,2020-02-01,No,Two year,No,Mailed check,80.85,NaN
1082,4367-NUYAO,2020-02-01,No,Two year,No,Mailed check,25.75,NaN
1340,1371-DWPAZ,2020-02-01,No,Two year,No,Credit card (automatic),56.05,NaN
3331,7644-OMVMY,2020-02-01,No,Two year,No,Mailed check,19.85,NaN
3826,3213-VVOLG,2020-02-01,No,Two year,No,Mailed check,25.35,NaN
4380,2520-SGTTA,2020-02-01,No,Two year,No,Mailed check,20.00,NaN
5218,2923-ARZLG,2020-02-01,No,One year,Yes,Mailed check,19.70,NaN
6670,4075-WKNIU,2020-02-01,No,Two year,No,Mailed check,73.35,NaN


Se observa que tenemos 11 datos NaN en la parte de 'TotalCharges' y es probable que sea porque sean clientes que apenas han aperturado su contrato. Se procede a eliminar esos 11 registros.

In [30]:
contract = contract.drop(contract[contract['TotalCharges'].isna()].index)
contract = contract.reset_index(drop=True)

In [31]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   BeginDate         7032 non-null   object 
 2   EndDate           7032 non-null   object 
 3   Type              7032 non-null   object 
 4   PaperlessBilling  7032 non-null   object 
 5   PaymentMethod     7032 non-null   object 
 6   MonthlyCharges    7032 non-null   float64
 7   TotalCharges      7032 non-null   float64
dtypes: float64(2), object(6)
memory usage: 439.6+ KB


Se realiza la unión de los dataframes para trabajar en 1 solo

In [32]:
df = contract.merge(internet, how='left', on='customerID')
df = df.merge(personal, how='left', on='customerID')
df = df.merge(phone, how='left', on='customerID')

df.head(10)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No
5,9305-CDSKC,2019-03-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,99.65,820.50,Fiber optic,No,No,Yes,No,Yes,Yes,Female,0,No,No,Yes
6,1452-KIOVK,2018-04-01,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,Fiber optic,No,Yes,No,No,Yes,No,Male,0,No,Yes,Yes
7,6713-OKOMC,2019-04-01,No,Month-to-month,No,Mailed check,29.75,301.90,DSL,Yes,No,No,No,No,No,Female,0,No,No,NaN
8,7892-POOKP,2017-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,104.80,3046.05,Fiber optic,No,No,Yes,Yes,Yes,Yes,Female,0,Yes,No,Yes
9,6388-TABGU,2014-12-01,No,One year,No,Bank transfer (automatic),56.15,3487.95,DSL,Yes,Yes,No,No,No,No,Male,0,No,Yes,No


Una vez con todos los dataframes juntos, se procede a adaptar los datos para ser ocupados en los modelos futuros.

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7031
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   BeginDate         7032 non-null   object 
 2   EndDate           7032 non-null   object 
 3   Type              7032 non-null   object 
 4   PaperlessBilling  7032 non-null   object 
 5   PaymentMethod     7032 non-null   object 
 6   MonthlyCharges    7032 non-null   float64
 7   TotalCharges      7032 non-null   float64
 8   InternetService   5512 non-null   object 
 9   OnlineSecurity    5512 non-null   object 
 10  OnlineBackup      5512 non-null   object 
 11  DeviceProtection  5512 non-null   object 
 12  TechSupport       5512 non-null   object 
 13  StreamingTV       5512 non-null   object 
 14  StreamingMovies   5512 non-null   object 
 15  gender            7032 non-null   object 
 16  SeniorCitizen     7032 non-null   int64  


In [34]:
df.isnull().sum()

customerID             0
BeginDate              0
EndDate                0
Type                   0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
InternetService     1520
OnlineSecurity      1520
OnlineBackup        1520
DeviceProtection    1520
TechSupport         1520
StreamingTV         1520
StreamingMovies     1520
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
MultipleLines        680
dtype: int64

De los datos faltantes:

phone

    - MultipleLines: el cliente no cuenta con servicio de telefonía. Los nulos se procederán a rellenar con "No"
    
internet

     - DataFrame completo: el cliente probablemente no cuente con servicio de internet, solo de telefonia
     
Por lo tanto, en los datos nulos del dataframe, se colocarán "0", como representación de 'no'

In [35]:
df = df.fillna(0)

In [36]:
df.isnull().sum()

customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
MultipleLines       0
dtype: int64

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7031
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   BeginDate         7032 non-null   object 
 2   EndDate           7032 non-null   object 
 3   Type              7032 non-null   object 
 4   PaperlessBilling  7032 non-null   object 
 5   PaymentMethod     7032 non-null   object 
 6   MonthlyCharges    7032 non-null   float64
 7   TotalCharges      7032 non-null   float64
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  gender            7032 non-null   object 
 16  SeniorCitizen     7032 non-null   int64  


In [38]:
df['exited'] = (df['EndDate'] != 'no').astype(int)

In [39]:
def to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [40]:
df.columns = [to_snake_case(col) for col in df.columns]

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7031
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7032 non-null   object 
 1   begin_date         7032 non-null   object 
 2   end_date           7032 non-null   object 
 3   type               7032 non-null   object 
 4   paperless_billing  7032 non-null   object 
 5   payment_method     7032 non-null   object 
 6   monthly_charges    7032 non-null   float64
 7   total_charges      7032 non-null   float64
 8   internet_service   7032 non-null   object 
 9   online_security    7032 non-null   object 
 10  online_backup      7032 non-null   object 
 11  device_protection  7032 non-null   object 
 12  tech_support       7032 non-null   object 
 13  streaming_tv       7032 non-null   object 
 14  streaming_movies   7032 non-null   object 
 15  gender             7032 non-null   object 
 16  senior_citizen     7032 